Step 1 : Simulate Rankings of Relevance for E and P (5 points)

In the first step you will generate pair s of rankings of relevance, for the production P and experimental E,
respectively, for a hypothetical query q . Assume a 3-graded relevance, i.e. {N, R, HR}. Construct all
possible P and E ranking pairs of length 5, for which E outperforms P.

Example:

P: {N N N N N}

E: {N N N N R}

…

P: {HR HR HR HR R}

E: {HR HR HR HR HR}

(Note 1: If you do not have enough computational power, sample 1000 pair uniformly at random to show
your work.)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import random as rand

In [34]:
#convert int to relevance
def to_string(_list): 
    L=[]
    for i in _list:
        if i==0:
            L.append("N ")
        elif i==1:
            L.append("R ")
        elif i==2:
            L.append("HR")
        else: 
            L.append("? ")
    return L
 
def outperforms(E,P):
    for i in range(len(E)):
        if E[i]<P[i]:
            return False
    return True

In [73]:
pairs = [] 
for p_possibility in range(243): #3^5 = 243 
    P=[None]*5
    for p_pos in range(5):
        P[p_pos]= p_possibility // (np.power(3,p_pos)) %3
    for e_possibility in range(243): #3^5 = 243 
        E=[None]*5
        for e_pos in range(5):
            E[e_pos]= e_possibility // (np.power(3,e_pos)) %3
        #if(outperforms(E,P)):
            #pairs.append((P,E))
        pairs.append((P,E))
               

In [74]:
print(len(pairs))
r = rand.randint(0,7775)
print("P: " , to_string(pairs[r][0]), "E: ",to_string(pairs[r][1]))

59049
P:  ['R ', 'R ', 'HR', 'N ', 'N '] E:  ['R ', 'HR', 'R ', 'R ', 'R ']


Step 2 : Implement Evaluation Measures (15 points)

Implement 1 binary and 2 multi-graded evaluation measures out of the 7 measures mentioned above.

(Note 2: Some of the aforementioned measures require the total number of relevant and highly relevant
documents in the entire collection – pay extra attention on how to find this)

TODO: ================================================================

========================================================================

===========================================================================

add if security if the number of relevant is null ! 

In [144]:
#TODO: choose 1 out of 3
#BINARY 
#Precision at rank r 
def Binary_measure_precision(testSet, r):
    num_test_rel = 0 
    for i in range(len(testSet)):
        if testSet[i]>0:
            if i<r:
                num_test_rel+=1
    return float(num_test_rel) / r

#Recall at rank k 
def Binary_measure_recal(testSet, r):
    num_test_rel = 0 
    total_num_rel=0
    for i in range(len(testSet)):
        if testSet[i]>0:
            if i<r:
                num_test_rel+=1
            total_num_rel+=1
    return float(num_test_rel) / total_num_rel

#Average precision at rank k 
def Binary_measure_avg_precision(testSet, r):
    num_rel=0
    sum_precision = 0 
    total_num_rel=0
    temp="("
    for i in range(len(testSet)):
        if testSet[i]>0:
            if i<r:
                num_rel+=1
                temp+=" "+str(num_rel)+"/"+str(i+1)+"+"
                sum_precision+=float(num_rel)/(i+1)
            total_num_rel+=1
    temp=temp[:len(temp)-1]+")"
    #print(temp,"/",total_num_rel)
    return float(sum_precision) / total_num_rel

#MULTI GRADED 
#Discount cumulative gain DCG at rank r
def DCG(testSet, r):
    res=0
    for k in range(1,r+1):#r starts at 1
        rel_r = testSet[k-1]
        res+=float(np.power(2,rel_r)-1)/(np.log2(1+k))
    return res

#Normalized discount cumulative gain nDCG at rank r
def nDCG(testSet, r):
    orderedSet= list(testSet)
    orderedSet.sort(reverse=True)
    res=0
    for k in range(1,r+1):
        res+=DCG(testSet, k) / DCG(orderedSet, k)
    return res/k
#Rank Biased Precision w ith persistence parameter 𝜃 =0.8
def RBP (testSet,theta=0.8):
    res=0
    for k in range(1,len(testSet)+1):
        rel_k=testSet[k-1]
        res+= rel_k * np.power(theta,k-1) * (1.0-theta)
    return res
        

In [145]:
test= [2,0,2,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1]#length 20, from example p 25 in Lecture 1b 
r=20
print('Precision',Binary_measure_precision(test,r))
print('Recal',Binary_measure_recal(test,r))
print('Average precision', Binary_measure_avg_precision(test,r))
print('DCG', DCG(test,r))
print('nDCG', nDCG(test,r))
print('RBP', RBP(test))

Precision 0.5
Recal 1.0
Average precision 0.6155128205128204
DCG 3.18738348176
nDCG 0.790962232212
RBP 0.897885306542


In [153]:
#precision , DCG, RBP 
outperf_pairs=[]
d_measure_precision=[]
d_measure_DCG=[]
d_measure_RBP=[]

rank = 4
for i in range(len(pairs)):#pairs made of (P,E)
    if(outperforms(pairs[i][1],pairs[i][0])):
        outperf_pairs.append(pairs[i])
        d_measure_precision.append(Binary_measure_precision(pairs[i][1],rank)-Binary_measure_precision(pairs[i][0],rank))
        d_measure_DCG.append(DCG(pairs[i][1],rank)-DCG(pairs[i][0],rank))
        d_measure_RBP.append(RBP(pairs[i][1])-RBP(pairs[i][0]))

#for i in range(len(outperf_pairs)):
    #print("P:",to_string(outperf_pairs[i][0]),"E:",to_string(outperf_pairs[i][1]))
    #print(d_measure_precision[i])
    #print(d_measure_DCG[i])
    #print(d_measure_RBP[i])

We have to specify what outperforms means: 
-E[rank]>P[rank] for all rank (I'll say either this one)
-E[rank]>P[rank] for a rank and for a specific measure
-E[rank]>P[rank] for all rank and for a specific measure
-E[rank]>P[rank] for all rank and for all specific measure (or this one)